In [1]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
## load mnist dataset
use_cuda = torch.cuda.is_available()
root = './data'
if not os.path.exists(root):
    os.mkdir(root)
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
# if not exist, download mnist dataset
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)
batch_size = 100
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

Processing...
Done!


In [ ]:
class DEPICT_AE(nn.Module):
    def __init__(self):
        super(DEPICT_AE,self).__init__()
        self.dropout = nn.Dropout(p=0.1)
        self.conv_ae1 = nn.Conv2d(1,50,4,stride=2,padding=2)
        self.conv_ae2 = nn.Conv2d(50,50,5,stride=2,padding=2)
        self.leReLU = nn.LeakyReLU()
        self.fc1 = nn.Linear(9*9*50,68)
        self.tanh = nn.Tanh()
        self.fc_de = nn.Linear(68,9*9*50)
        self.conv_de2 = nn.ConvTranspose2d(50,50,5,stride=2,padding=2)
        self.conv_de1 = nn.ConvTranspose2d(50,1,4,stride=2,padding=2)
    def forward(self,x):
        # 32x32x1
        x = self.dropout(x)
        # 32x32x1
        x = self.conv_ae1(x)
        # 17x17x50
        x = self.leReLU(x)
        # 17x17x50
        x = self.dropout(x)
        # 17x17x50
        x = self.conv_ae2(x)
        # 9x9x50
        x = self.leReLU(x)
        # 9x9x50
        x = self.dropout(x)
        # 9x9x50
        x = x.view(-1, 9*9*50)
        # 1x4050
        x = self.fc1(x)
        # 1x68
        x = self.tanh(x)
        # 1x68
        ##### auto encoder is done, followed by decoder #####
        # 1x68
        x = self.fc_de(x)
        # 1x4050
        x = self.tanh(x)
        # 1x4050
        x = x.view(-1,9,9,50)
        # 9*9*50
        x = x.conv_de2(x)
        # 17x17x50
        x = self.leReLU(x)
        # 17x17x50
        x = self.conv_de1(x)
        # 32x32x1
        x = self.tanh(x)
        
        
        
        
        
        
        

In [2]:
x = torch.randn(1,4050)
y = x.view(-1,9,9,50)
y.size()

torch.Size([1, 9, 9, 50])